# Reliability_testing
## DV
* 因素分析 -> 根據構面取平均 : online_media_pp_mean / voting_mean / offline_media_pp_mean / campaign_worker_pp_mean
* 因素分析 -> 根據構面取綜合得分 : online_media_pp_score / voting_score / offline_media_pp_score / campaign_worker_pp_score
* 先分 online_pp / offline_pp -> 因素分析 -> 根據因素分析結果建立兩個綜合得分 : onlnie_scores / offlnie_scores
## anti_party
* 因素分析 -> 根據構面取平均 : political_polarization_mean / party_image_mean
* 因素分析 -> 根據構面取綜合得分 : political_polarization_score / party_image_score
* 因素分析 -> 全部算一個綜合得分 : anti_party_scores



## Import Package

In [1]:
# ! pip install factor_analyzer pingouin

In [2]:
# for data ETL
import pandas as pd
import ETL
# reliability testing
from factor_analyzer import FactorAnalyzer
import pingouin as pg 

## Load data and Data Cleaning

In [3]:
raw_data = pd.read_csv('raw_data.csv')

In [4]:
filter_data = raw_data.loc[raw_data['目前國內政黨當中，請問您是否偏向哪一個政黨？'].isin(['沒有特定支持', '都不支持'])]
filter_data.drop(['您有絶對的權力決定是否要參與本研究。若您願意參與，請務必勾選下列選項：', '請填寫您的電子信箱，以利後續抽獎聯繫使用'], axis=1, inplace=True)

/var/folders/dn/wbyhv8k14697mm1zy19w1jh40000gn/T/ipykernel_94883/2097987034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data.drop(['您有絶對的權力決定是否要參與本研究。若您願意參與，請務必勾選下列選項：', '請填寫您的電子信箱，以利後續抽獎聯繫使用'], axis=1, inplace=True)


In [5]:
ml_df = ETL.data_cleaning(filter_data)
ml_df = ml_df.reset_index(drop=True)

/Users/wen/Desktop/github/media_political/ETL.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data['TV_news_time'] = filter_data['請問您平均每天花多少時間注意電視上的選舉新聞？'].apply(ETL_function.news_time)
/Users/wen/Desktop/github/media_political/ETL.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data['news_paper_time'] = filter_data['請問您平均每天花多少時間注意報紙上的選舉新聞？'].apply(ETL_function.news_time)
/Users/wen/Desktop/github/media_political/ETL.py:16: SettingWithCopyWarning: 
A value is trying to be set on a co

In [6]:
ml_df = ETL.feature_engineering(ml_df)
ml_df = ETL.DV_feature(ml_df)
ml_df = ETL.anti_party_feature(ml_df)
ml_df.shape

(503, 51)

In [7]:
ml_df.columns

Index(['TV_news_time', 'news_paper_time', 'int_news_time', 'TV_debate',
       'read_media', 'like_media', 'share_media', 'comment_media',
       'int_discuss', 'read_election_news', 'read_election_leaflet',
       'convince', 'campaign', 'volunteer', 'election_mayor', 'election_18',
       'sex', 'ethnic', 'edu', 'income', 'pk_1', 'pk_2', 'pk_3',
       'political_knowledge', 'anti_1', 'anti_2', 'anti_3', 'anti_4', 'anti_5',
       'TC_issue', 'Negative_1', 'Negative_2', 'Negative_3', 'online_pp',
       'offline_pp', 'anti_party', 'online_media_pp_mean', 'voting_mean',
       'offline_media_pp_mean', 'campaign_worker_pp_mean',
       'online_media_pp_score', 'voting_score', 'offline_media_pp_score',
       'campaign_worker_pp_score', 'onlnie_scores', 'offlnie_scores',
       'political_polarization_mean', 'party_image_mean',
       'political_polarization_score', 'party_image_score',
       'anti_party_scores'],
      dtype='object')

## Reliability Testing

## DV testing

### Basic

In [8]:
online_pp_vars = ['online_pp', 'TV_news_time', 'news_paper_time', 'int_news_time', 'TV_debate',                   'read_media', 'like_media', 'share_media', 'comment_media', 'int_discuss']
offline_pp_vars = ['offline_pp', 'read_election_news', 'read_election_leaflet', 'convince',                        'campaign', 'volunteer', 'election_mayor', 'election_18']
online_pp_tesing_df = ml_df[online_pp_vars]
offline_pp_tesing_df = ml_df[offline_pp_vars]

In [9]:
alpha_1 = pg.cronbach_alpha(online_pp_tesing_df)
alpha_2 = pg.cronbach_alpha(offline_pp_tesing_df)

print('online_pp Cronbach\'s alpha: ', alpha_1[0])
print('offline_pp Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

online_pp Cronbach's alpha:  0.7717699094188335
offline_pp Cronbach's alpha:  0.809844435651358
Total Cronbanc's alpha: 0.7908071725350958


### Mean after FAC

In [10]:
online_media_pp_vars_mean = ['read_media', 'like_media', 'share_media', 'comment_media', 'online_media_pp_mean']
online_media_mean_tesing_df = ml_df[online_media_pp_vars_mean]
voting_vars_mean = ['election_mayor', 'election_18', 'voting_mean']
voting_mean_tesing_df = ml_df[voting_vars_mean]
offline_media_pp_vars_mean = ['read_election_news', 'read_election_leaflet', 'offline_media_pp_mean']
offline_media_mean_tesing_df = ml_df[offline_media_pp_vars_mean]
campaign_worker_pp_vars_mean = ['campaign', 'volunteer', 'campaign_worker_pp_mean']
campaign_worker_mean_tesing_df = ml_df[campaign_worker_pp_vars_mean]

In [11]:
alpha_1 = pg.cronbach_alpha(online_media_mean_tesing_df)
alpha_2 = pg.cronbach_alpha(voting_mean_tesing_df)
alpha_3 = pg.cronbach_alpha(offline_media_mean_tesing_df)
alpha_4 = pg.cronbach_alpha(campaign_worker_mean_tesing_df)

print('online_media_pp_mean Cronbach\'s alpha: ', alpha_1[0])
print('voting_mean Cronbach\'s alpha: ', alpha_2[0])
print('offline_media_pp_mean Cronbach\'s alpha: ', alpha_3[0])
print('campaign_worker_pp_mean Cronbach\'s alpha: ', alpha_4[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0] + alpha_3[0] + alpha_4[0])/4)

online_media_pp_mean Cronbach's alpha:  0.8150949007062644
voting_mean Cronbach's alpha:  0.9821418184416824
offline_media_pp_mean Cronbach's alpha:  0.9267895495036799
campaign_worker_pp_mean Cronbach's alpha:  0.9058515655096374
Total Cronbanc's alpha: 0.907469458540316


### Total score after FAC

In [12]:
online_media_pp_vars_score = ['read_media', 'like_media', 'share_media', 'comment_media', 'online_media_pp_score']
online_media_score_tesing_df = ml_df[online_media_pp_vars_score]
voting_vars_score = ['election_mayor', 'election_18', 'voting_score']
voting_score_tesing_df = ml_df[voting_vars_score]
offline_media_pp_vars_score = ['read_election_news', 'read_election_leaflet', 'offline_media_pp_score']
offline_media_score_tesing_df = ml_df[offline_media_pp_vars_score]
campaign_worker_pp_vars_score = ['campaign', 'volunteer', 'campaign_worker_pp_score']
campaign_worker_score_tesing_df = ml_df[campaign_worker_pp_vars_score]

In [13]:
alpha_1 = pg.cronbach_alpha(online_media_score_tesing_df)
alpha_2 = pg.cronbach_alpha(voting_score_tesing_df)
alpha_3 = pg.cronbach_alpha(offline_media_score_tesing_df)
alpha_4 = pg.cronbach_alpha(campaign_worker_mean_tesing_df)

print('online_media_pp_score Cronbach\'s alpha: ', alpha_1[0])
print('voting_score Cronbach\'s alpha: ', alpha_2[0])
print('offline_media_pp_score Cronbach\'s alpha: ', alpha_3[0])
print('campaign_worker_pp_score Cronbach\'s alpha: ', alpha_4[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0] + alpha_3[0] + alpha_4[0])/4)

online_media_pp_score Cronbach's alpha:  0.8311674953424751
voting_score Cronbach's alpha:  0.9666132110606549
offline_media_pp_score Cronbach's alpha:  0.8956567852370945
campaign_worker_pp_score Cronbach's alpha:  0.9058515655096374
Total Cronbanc's alpha: 0.8998222642874656


### Grouping and FAC get total score

In [14]:
online_pp_vars = ['onlnie_scores', 'TV_news_time', 'news_paper_time', 'int_news_time', 'TV_debate','read_media', 'like_media', 'share_media', 'comment_media', 'int_discuss']
offline_pp_vars = ['offlnie_scores', 'read_election_news', 'read_election_leaflet', 'convince',                    'campaign', 'volunteer', 'election_mayor', 'election_18']
online_pp_tesing_df = ml_df[online_pp_vars]
offline_pp_tesing_df = ml_df[offline_pp_vars]

In [15]:
alpha_1 = pg.cronbach_alpha(online_pp_tesing_df)
alpha_2 = pg.cronbach_alpha(offline_pp_tesing_df)

print('onlnie_scores Cronbach\'s alpha: ', alpha_1[0])
print('offlnie_scores Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

onlnie_scores Cronbach's alpha:  0.7865005681885463
offlnie_scores Cronbach's alpha:  0.8136118677407488
Total Cronbanc's alpha: 0.8000562179646475


## anti_party testing

### Basic

In [16]:
anti_party_vars = ['anti_party', 'anti_1', 'anti_2', 'anti_3', 'anti_4', 'anti_5']
anti_party_tesing_df = ml_df[anti_party_vars]

In [17]:
alpha = pg.cronbach_alpha(anti_party_tesing_df)

print('anti_party Cronbach\'s alpha: ', alpha[0])

anti_party Cronbach's alpha:  0.7564819668378752


### Mean after FAC

In [18]:
political_polarization_mean_vars = ['anti_1', 'political_polarization_mean']
party_image_mean_vars = ['anti_3', 'anti_4', 'anti_5', 'party_image_mean']
political_polarization_mean_tesing_df = ml_df[political_polarization_mean_vars]
party_image_mean_vars_tesing_df = ml_df[party_image_mean_vars]

In [19]:
alpha_1 = pg.cronbach_alpha(political_polarization_mean_tesing_df)
alpha_2 = pg.cronbach_alpha(party_image_mean_vars_tesing_df)

print('political_polarization_mean Cronbach\'s alpha: ', alpha_1[0])
print('party_image_mean Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

political_polarization_mean Cronbach's alpha:  1.0
party_image_mean Cronbach's alpha:  0.8485172366992396
Total Cronbanc's alpha: 0.9242586183496198


### Total score after FAC two group

In [20]:
political_polarization_score_vars = ['anti_1', 'political_polarization_score']
party_image_score_vars = ['anti_3', 'anti_4', 'anti_5', 'party_image_score']
political_polarization_score_tesing_df = ml_df[political_polarization_score_vars]
party_image_score_tesing_df = ml_df[party_image_score_vars]

In [21]:
alpha_1 = pg.cronbach_alpha(political_polarization_score_tesing_df)
alpha_2 = pg.cronbach_alpha(party_image_score_tesing_df)

print('political_polarization_mean Cronbach\'s alpha: ', alpha_1[0])
print('party_image_mean Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

political_polarization_mean Cronbach's alpha:  0.1487612521856283
party_image_mean Cronbach's alpha:  0.5807386999310291
Total Cronbanc's alpha: 0.3647499760583287


### Total score after FAC

In [22]:
anti_party_vars = ['anti_party_scores', 'anti_1', 'anti_2', 'anti_3', 'anti_4', 'anti_5']
anti_party_tesing_df = ml_df[anti_party_vars]

In [23]:
alpha = pg.cronbach_alpha(anti_party_tesing_df)

print('anti_party Cronbach\'s alpha: ', alpha[0])

anti_party Cronbach's alpha:  0.7405711986436132
